<a href="https://colab.research.google.com/github/luizaclara/KNNtitanic/blob/main/Trabalho_02ICD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/c/titanic/data

https://github.com/datasciencedojo/datasets/blob/master/titanic.csv

PassengerId
Survived
Pclass
Name
Sex
Age
SibSp
Parch
Ticket
Fare
Cabin
Embarked

In [1]:
import sklearn
import pandas as pd
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, accuracy_score


In [2]:
class KNN ():
  def __init__(self,k=3):
    self.k = k

  def fit(self,X,Y):
    self.X = X
    self.Y = Y

  def distance(self, point):
    return list(zip(euclidean_distances(self.X, [point]), self.Y))

  def predict_one(self,x):
    distances = sorted(self.distance(x),lambda d:d [0])[:self.k]

    tags_count = {} # {tag : (contagem , soma das distancias)}
    
    for dist, tag in distances:
      if tag not in tags_count:
        count[tag] = (0, 0)
      count[tag] = (count[tag][0] + 1, count[tag][1] + dist )

    # ordena em ordem decrescente a contagem e em ordem crescente a distância
    sorted_count = sorted(tags_count.items(), reverse = True, key= lambda x: (x[1][0], -x[1][1]))
    
    return sorted_count[0][0] # nome da categoria com mais contagem e menos distância

  def predict(self, points):
    return [self.predict_one(p) for p in points]


In [3]:
dataframe = pd.read_csv("titanic.csv")

In [4]:
# Normalizar
for column in ['Age', 'Fare']:
    dataframe[column] = dataframe[column]  / dataframe[column].abs().max() 

In [5]:
# Transformar a coluna 'Sex' e 'Embarked' em duas colunas
encoder = OneHotEncoder(sparse_output=False)
encoded_data = encoder.fit_transform(dataframe[['Sex', 'Embarked']])
encoded_dataframe = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['Sex', 'Embarked']))

In [6]:
# juntar os dataframes
dataframe = pd.concat([dataframe, encoded_dataframe])

In [7]:
# substituir valores nulos

# preenche a idade com a média das idades
dataframe['Age'] = dataframe['Age'].fillna(dataframe['Age'].mean()) 

#preenche o sexo com o que aparece mais vezes
# dataframe['Sex'] = dataframe['Sex'].fillna(dataframe['Sex'].mode())

# drop no 'Embarked' porque tem poucos valores NaN
dataframe.dropna(subset='Embarked', inplace=True)

In [8]:
columns = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'] + list(encoded_dataframe.columns)
y = ['Survived']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(dataframe[[col for col in columns]], dataframe[y], test_size=0.25, random_state=42)

In [ ]:
values = []
for k in range(1, 30, 4):
    knn = KNN(k)
    knn.fit(X_train, y_train)
    
    y_pred = knn.predict(X_test)
    conf_matrix = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    values.append((accuracy, k))